In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import yaml
from ultralytics import YOLO
import onnxruntime as ort
from openvino.runtime import Core
import ncnn
import time
import random
import torch


2025-11-17 00:02:19.144123005 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
/home/pi5/TrafficSign/venv_tsr/lib/python3.13/site-packages/openvino/runtime/__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


In [2]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114)):
    """Resize and pad image to new_shape with letterbox"""
    shape = img.shape[:2]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)
    
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    
    dw /= 2
    dh /= 2
    
    if shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    return img, r, (dw, dh)


def nms_numpy(boxes, scores, iou_threshold=0.45):
    """Non-Maximum Suppression - DÙNG CHUNG CHO TẤT CẢ"""
    if len(boxes) == 0:
        return []
    
    x1, y1, x2, y2 = boxes.T
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    keep = []

    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        if len(order) == 1:
            break
            
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter + 1e-6)

        inds = np.where(iou <= iou_threshold)[0]
        order = order[inds + 1]

    return keep


def preprocess_image(img, input_size=(640, 640)):
    """
    Preprocess CHUNG cho ONNX/OpenVINO/NCNN (numpy version)
    Trả về numpy array để fair
    """
    img_resized, ratio, (dw, dh) = letterbox(img, new_shape=input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_normalized = img_rgb.astype(np.float32) / 255.0
    img_transposed = img_normalized.transpose(2, 0, 1)
    img_batch = np.expand_dims(img_transposed, axis=0)
    
    return img_batch, ratio, (dw, dh)


def postprocess_with_nms(output, orig_shape, ratio, pad, conf_threshold=0.25, iou_threshold=0.45):
    """
    Postprocess CHUNG CHO TẤT CẢ BACKEND (có NMS)
    Đảm bảo tính công bằng
    """
    # Normalize shape to (1, 84, 8400)
    if isinstance(output, ncnn.Mat):
        output = np.array(output)
    
    if output.ndim == 2:
        output = np.expand_dims(output, axis=0)
    if output.shape[-1] == 84:
        output = output.transpose(0, 2, 1)
    
    predictions = output[0]  # (84, 8400)
    boxes = predictions[:4].T  # (8400, 4)
    scores = predictions[4:].T  # (8400, num_classes)
    
    # Get max score and class
    class_scores = np.max(scores, axis=1)
    class_ids = np.argmax(scores, axis=1)
    
    # Filter by confidence
    mask = class_scores > conf_threshold
    boxes = boxes[mask]
    scores = class_scores[mask]
    class_ids = class_ids[mask]
    
    if len(boxes) == 0:
        return np.empty((0, 4)), np.empty((0,)), np.empty((0,))
    
    # Convert xywh to xyxy
    x_center, y_center, width, height = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    x1 = x_center - width / 2
    y1 = y_center - height / 2
    x2 = x_center + width / 2
    y2 = y_center + height / 2
    boxes_xyxy = np.stack([x1, y1, x2, y2], axis=1)
    
    # Scale back to original image
    boxes_xyxy[:, [0, 2]] -= pad[0]
    boxes_xyxy[:, [1, 3]] -= pad[1]
    boxes_xyxy /= ratio
    
    # Clip to image boundaries
    boxes_xyxy[:, [0, 2]] = np.clip(boxes_xyxy[:, [0, 2]], 0, orig_shape[1])
    boxes_xyxy[:, [1, 3]] = np.clip(boxes_xyxy[:, [1, 3]], 0, orig_shape[0])
    
    # Apply NMS per class
    nms_indices = []
    for cls in np.unique(class_ids):
        cls_mask = class_ids == cls
        keep = nms_numpy(boxes_xyxy[cls_mask], scores[cls_mask], iou_threshold)
        nms_indices.extend(np.where(cls_mask)[0][keep])
    
    if len(nms_indices) > 0:
        nms_indices = np.array(nms_indices)
        boxes_xyxy = boxes_xyxy[nms_indices]
        scores = scores[nms_indices]
        class_ids = class_ids[nms_indices]
    else:
        boxes_xyxy = np.empty((0, 4))
        scores = np.empty((0,))
        class_ids = np.empty((0,))
    
    return boxes_xyxy, scores, class_ids



# EVAL PYTORCH

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [4]:
def benchmark_pytorch_inference(
    model_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    iou_threshold=0.45,
    device='cpu'
):
    
    """Benchmark PyTorch model với cùng preprocessing/postprocessing"""
    print(f"\n🚀 Benchmarking PyTorch: {Path(model_path).name}")
    print(f"📐 Input size: {input_size}")
    print(f"🖥️  Device: {device}")
    print(f"📊 Running {num_samples} samples (after {num_warmup} warmup)\n")
    
    # Load model
    model = YOLO(model_path)
    model.to(device)
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_config.get('val', 'valid/images')
    image_files = list(val_path.glob('*.jpg')) + list(val_path.glob('*.png'))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
        
    # Warmup
    print(f"🔥 Warmup ({num_warmup} runs)...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        input_tensor, _, _ = preprocess_image(img, input_size=input_size)
        input_torch = torch.from_numpy(input_tensor).to(device)
        with torch.no_grad():
            _ = model.model(input_torch)
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running inference on {len(sample_images)} test images...")
    for img_path in sample_images:
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        input_tensor, ratio, pad = preprocess_image(img, input_size=input_size)
        input_torch = torch.from_numpy(input_tensor).to(device)
        t1 = time.time()
        
        # Inference
        with torch.no_grad():
            outputs = model.model(input_torch)
        t2 = time.time()
        
        # Postprocess
        output_np = outputs[0].cpu().numpy() if isinstance(outputs[0], torch.Tensor) else outputs[0]
        boxes, scores, class_ids = postprocess_with_nms(
            output_np, (orig_h, orig_w), ratio, pad, conf_threshold, iou_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS")
    
    return {
        "Model": model_name,
        "Backend": f"PyTorch ({device.upper()})",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }


In [5]:
def benchmark_onnx_inference(
    onnx_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    iou_threshold=0.45,
    device_provider='CPUExecutionProvider'
):
    """Benchmark ONNX với postprocess GIỐNG HỆT các backend khác"""
    print(f"\n🚀 Benchmarking ONNX: {Path(onnx_path).name}")
    print(f"⚙️  Provider: {device_provider}")
    print(f"📐 Input size: {input_size}")
    print(f"📊 Running {num_samples} images (after {num_warmup} warmup)")
    
    # Load model
    session = ort.InferenceSession(onnx_path, providers=[device_provider])
    input_name = session.get_inputs()[0].name
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_config.get('val', 'valid/images')
    image_files = list(val_path.glob('*.jpg')) + list(val_path.glob('*.png'))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    # Warmup
    print(f"\n🔥 Warmup ({num_warmup} runs)...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        input_tensor, _, _ = preprocess_image(img, input_size=input_size)
        _ = session.run(None, {input_name: input_tensor})
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running inference on {len(sample_images)} test images...")
    for img_path in sample_images:
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        input_tensor, ratio, pad = preprocess_image(img, input_size=input_size)
        t1 = time.time()
        
        # Inference
        outputs = session.run(None, {input_name: input_tensor})
        t2 = time.time()
        
        # Postprocess (DÙNG HÀM CHUNG)
        boxes, scores, class_ids = postprocess_with_nms(
            outputs[0], (orig_h, orig_w), ratio, pad, conf_threshold, iou_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS")
    
    return {
        "Model": model_name,
        "Backend": "ONNX (CPU)",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }


In [6]:
def benchmark_openvino_inference(
    model_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    iou_threshold=0.45,
    device="CPU"
):
    """Benchmark OpenVINO với postprocess GIỐNG HỆT các backend khác"""
    print(f"\n🚀 Benchmarking OpenVINO: {Path(model_path).name}")
    print(f"📐 Input size: {input_size}")
    print(f"🖥️  Device: {device}")
    print(f"📊 Running {num_samples} samples (after {num_warmup} warmup)\n")
    
    # Load model
    ie = Core()
    model = ie.read_model(model=model_path)
    compiled_model = ie.compile_model(model=model, device_name=device)
    input_layer = compiled_model.input(0)
    output_layer = compiled_model.output(0)
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_cfg = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_cfg.get("val", "valid/images")
    image_files = list(val_path.glob("*.jpg")) + list(val_path.glob("*.png"))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    # Warmup
    print(f"🔥 Warming up for {num_warmup} runs...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        input_tensor, _, _ = preprocess_image(img, input_size=input_size)
        _ = compiled_model([input_tensor])[output_layer]
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running timed inference on {len(sample_images)} test images...")
    for img_path in sample_images:
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        input_tensor, ratio, pad = preprocess_image(img, input_size=input_size)
        t1 = time.time()
        
        # Inference
        outputs = compiled_model([input_tensor])[output_layer]
        t2 = time.time()
        
        # Postprocess (DÙNG HÀM CHUNG)
        boxes, scores, class_ids = postprocess_with_nms(
            outputs, (orig_h, orig_w), ratio, pad, conf_threshold, iou_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS")
    
    return {
        "Model": model_name,
        "Backend": "OpenVINO (CPU)",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }


In [7]:
def preprocess_image_ncnn(img, input_size=(640, 640)):
    """
    Preprocess cho NCNN - convert sang ncnn.Mat từ numpy array
    Để fair, ta vẫn dùng letterbox giống các backend khác
    """
    # Dùng letterbox chung
    img_resized, ratio, (dw, dh) = letterbox(img, new_shape=input_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    
    # Convert to ncnn.Mat
    mat_in = ncnn.Mat.from_pixels(
        img_rgb,
        ncnn.Mat.PixelType.PIXEL_RGB,
        img_rgb.shape[1],
        img_rgb.shape[0]
    )
    
    # Normalize
    mean_vals = [0, 0, 0]
    norm_vals = [1/255.0, 1/255.0, 1/255.0]
    mat_in.substract_mean_normalize(mean_vals, norm_vals)
    
    return mat_in, ratio, (dw, dh)


def benchmark_ncnn_inference(
    param_path,
    bin_path,
    data_yaml,
    model_name,
    input_size=(640, 640),
    num_warmup=5,
    num_samples=50,
    conf_threshold=0.25,
    iou_threshold=0.45,
    input_name="in0",
    output_name="out0",
    use_gpu=False,
    num_threads=4,
):
    """Benchmark NCNN với postprocess GIỐNG HỆT các backend khác"""
    print(f"\n🚀 Benchmarking NCNN: {Path(param_path).stem}")
    print(f"📦 Model: {param_path}")
    print(f"🧩 Device: {'GPU (Vulkan)' if use_gpu else f'CPU ({num_threads} threads)'}")
    print(f"📐 Input size: {input_size}")
    print(f"📊 Running {num_samples} images (after {num_warmup} warmup)\n")
    
    # Load model
    net = ncnn.Net()
    net.opt.use_vulkan_compute = use_gpu
    net.opt.num_threads = num_threads
    
    if net.load_param(param_path) != 0:
        raise RuntimeError(f"Failed to load param: {param_path}")
    if net.load_model(bin_path) != 0:
        raise RuntimeError(f"Failed to load bin: {bin_path}")
    
    # Load dataset
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    dataset_root = Path(data_yaml).parent
    val_path = dataset_root / data_config.get('val', 'valid/images')
    image_files = list(val_path.glob('*.jpg')) + list(val_path.glob('*.png'))
    
    if len(image_files) == 0:
        raise ValueError(f"No images found in {val_path}")
    
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    # Warmup
    print(f"🔥 Warmup ({num_warmup} runs)...")
    for _ in range(num_warmup):
        img = cv2.imread(str(random.choice(sample_images)))
        mat_in, _, _ = preprocess_image_ncnn(img, input_size=input_size)
        ex = net.create_extractor()
        ex.input(input_name, mat_in)
        ex.extract(output_name)
    
    # Benchmark
    preprocess_times, inference_times, postprocess_times = [], [], []
    
    print(f"\n⏱️  Running inference on {len(sample_images)} test images...")
    for img_path in sample_images:
        img = cv2.imread(str(img_path))
        orig_h, orig_w = img.shape[:2]
        
        # Preprocess
        t0 = time.time()
        mat_in, ratio, pad = preprocess_image_ncnn(img, input_size=input_size)
        t1 = time.time()
        
        # Inference
        ex = net.create_extractor()
        ex.input(input_name, mat_in)
        ret, mat_out = ex.extract(output_name)
        t2 = time.time()
        
        if ret != 0:
            print(f"⚠️  Failed inference on {img_path.name}")
            continue
        
        # Postprocess (DÙNG HÀM CHUNG)
        output_array = np.array(mat_out)
        boxes, scores, class_ids = postprocess_with_nms(
            output_array, (orig_h, orig_w), ratio, pad, conf_threshold, iou_threshold
        )
        t3 = time.time()
        
        preprocess_times.append(t1 - t0)
        inference_times.append(t2 - t1)
        postprocess_times.append(t3 - t2)
    
    # Summary
    avg_pre = np.mean(preprocess_times) * 1000
    avg_inf = np.mean(inference_times) * 1000
    avg_post = np.mean(postprocess_times) * 1000
    total_ms = avg_pre + avg_inf + avg_post
    fps = 1000 / total_ms
    
    print(f"\n📊 Average Timing (over {len(sample_images)} images):")
    print(f"   🧩 Preprocess: {avg_pre:.2f} ms")
    print(f"   ⚙️  Inference : {avg_inf:.2f} ms")
    print(f"   📦 Postprocess: {avg_post:.2f} ms")
    print(f"   ⏱️  Total: {total_ms:.2f} ms → {fps:.2f} FPS\n")
    
    device_name = "GPU (Vulkan)" if use_gpu else f"CPU ({num_threads}t)"
    
    return {
        "Model": model_name,
        "Backend": f"NCNN ({device_name})",
        "Avg Preprocess (ms)": avg_pre,
        "Avg Inference (ms)": avg_inf,
        "Avg Postprocess (ms)": avg_post,
        "Total (ms)": total_ms,
        "FPS": fps,
    }


### RUN EVAL FPS PYTORCH ONNX

In [8]:

# Dataset path
data_yaml = "./Dataset/Detect/data_detect_vntsr/data.yaml"

# Model paths
models_pt = {
    "YOLOv5n": "./weight/yolo5.pt",
    "YOLOv8n": "./weight/yolo8.pt",
    "YOLO11n": "./weight/yolo11.pt",
    "yolo_custom": "./weight/yolo_plus/yolo_plus_v2.pt",
}

models_onnx = {
    "YOLOv5n": "./convert/model/yolo5/yolo5.onnx",
    "YOLOv8n": "./convert/model/yolo8/yolo8.onnx",
    "YOLO11n": "./convert/model/yolo11/yolo11.onnx",
    "yolo_custom": "./convert/model/yolo_plus/yolo_plus.onnx",

}

# models_openvino = {
#     "YOLOv5n": "./convert/model/yolov5/yolo5_openvino_model/yolo5.xml",
#     "YOLOv8n": "./convert/model/yolov8/yolo8_openvino_model/yolo8.xml",
#     "YOLO11n": "./convert/model/yolov11/yolo11_openvino_model/yolo11.xml",
#     "yolo_custom": "./convert/model/yolo_plus/yolo_plus_openvino_model/yolo_plus.xml",
# }

# models_ncnn = {
#     "YOLOv5n": "./convert/model/yolov5/yolo5_ncnn_model",
#     "YOLOv8n": "./convert/model/yolov8/yolo8_ncnn_model",
#     "YOLO11n": "./convert/model/yolov11/yolo11_ncnn_model",
#     "yolo_custom": "./convert/model/yolo_plus/yolo_plus_ncnn_model",
# }

results = []

# %%
# --- Benchmark PyTorch ---
print("\n" + "="*60)
print("🔥 PYTORCH BENCHMARK")
print("="*60)

for model_name, model_path in models_pt.items():
    try:
        result = benchmark_pytorch_inference(
            model_path=model_path,
            data_yaml=data_yaml,
            model_name=model_name,
            input_size=(640, 640),
            num_warmup=5,
            num_samples=100,
            device='cpu'
        )
        results.append(result)
    except Exception as e:
        print(f"Error benchmarking PyTorch {model_name}: {e}")

# %%
# --- Benchmark ONNX ---
print("\n" + "="*60)
print("🔥 ONNX BENCHMARK")
print("="*60)

for model_name, model_path in models_onnx.items():
    try:
        result = benchmark_onnx_inference(
            onnx_path=model_path,
            data_yaml=data_yaml,
            model_name=model_name,
            input_size=(640, 640),
            num_warmup=5,
            num_samples=100,
            device_provider="CPUExecutionProvider"
        )
        results.append(result)
    except Exception as e:
        print(f"Error benchmarking ONNX {model_name}: {e}")
# # --- Benchmark OpenVINO ---
# print("\n" + "="*60)
# print("🔥 OPENVINO BENCHMARK")
# print("="*60)

# for model_name, model_path in models_openvino.items():
#     try:
#         result = benchmark_openvino_inference(
#             model_path=model_path,
#             data_yaml=data_yaml,
#             model_name=model_name,
#             input_size=(640, 640),
#             num_warmup=5,
#             num_samples=100,
#             device="CPU"
#         )
#         if result:
#             results.append(result)
#     except Exception as e:
#         print(f"❌ Error benchmarking OpenVINO {model_name}: {e}")

# # %%
# # --- Benchmark NCNN ---
# print("\n" + "="*60)
# print("🔥 NCNN BENCHMARK")
# print("="*60)

# for model_name, model_dir in models_ncnn.items():
#     try:
#         param_path = os.path.join(model_dir, "model.ncnn.param")
#         bin_path = os.path.join(model_dir, "model.ncnn.bin")
        
#         if not (os.path.exists(param_path) and os.path.exists(bin_path)):
#             print(f"⚠️  Missing NCNN model files for {model_name}, skipping.")
#             continue
        
#         result = benchmark_ncnn_inference(
#             param_path=param_path,
#             bin_path=bin_path,
#             data_yaml=data_yaml,
#             model_name=model_name,
#             input_size=(640, 640),
#             num_warmup=5,
#             num_samples=100,
#             use_gpu=False,
#             num_threads=4
#         )
#         if result:
#             results.append(result)
#     except Exception as e:
#         print(f"❌ Error benchmarking NCNN {model_name}: {e}")


print("\n" + "="*60)
print("✅ BENCHMARK SUMMARY")
print("="*60)

df = pd.DataFrame(results)

print(df.to_string(index=False))

# Save to CSV
save_path = "./Eval/FPS/detect_eval_results_fps_v5_8_11_custom.csv"
df.to_csv(save_path, index=False)
print(f"\n💾 Saved results to: {save_path}")


🔥 PYTORCH BENCHMARK

🚀 Benchmarking PyTorch: yolo5.pt
📐 Input size: (640, 640)
🖥️  Device: cpu
📊 Running 100 samples (after 5 warmup)

🔥 Warmup (5 runs)...

⏱️  Running inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 5.31 ms
   ⚙️  Inference : 396.28 ms
   📦 Postprocess: 0.47 ms
   ⏱️  Total: 402.05 ms → 2.49 FPS

🚀 Benchmarking PyTorch: yolo8.pt
📐 Input size: (640, 640)
🖥️  Device: cpu
📊 Running 100 samples (after 5 warmup)

🔥 Warmup (5 runs)...

⏱️  Running inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 5.14 ms
   ⚙️  Inference : 422.35 ms
   📦 Postprocess: 0.47 ms
   ⏱️  Total: 427.96 ms → 2.34 FPS

🚀 Benchmarking PyTorch: yolo11.pt
📐 Input size: (640, 640)
🖥️  Device: cpu
📊 Running 100 samples (after 5 warmup)

🔥 Warmup (5 runs)...

⏱️  Running inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 5.18 ms
   ⚙️  Inference : 397.87 ms
   📦 Postprocess: 0.47 ms
   ⏱️  Total: 

In [9]:
models_openvino = {
    "YOLO5n": "./convert/model/yolo5/yolo5_openvino_model/yolo5.xml",
    "YOLO8n": "./convert/model/yolo8/yolo8_openvino_model/yolo8.xml",
    "YOLO11n": "./convert/model/yolo11/yolo11_openvino_model/yolo11.xml",
    "yolo_custom": "./convert/model/yolo_plus/yolo_plus_openvino_model/yolo_plus.xml",
}

models_ncnn = {
    "YOLO5n": "./convert/model/yolo5/yolo5_ncnn_model",
    "YOLO8n": "./convert/model/yolo8/yolo8_ncnn_model",
    "YOLO11n": "./convert/model/yolo11/yolo11_ncnn_model",
    "yolo_custom": "./convert/model/yolo_plus/yolo_plus_ncnn_model",
}

results = []
# --- Benchmark OpenVINO ---
print("\n" + "="*60)
print("🔥 OPENVINO BENCHMARK")
print("="*60)

for model_name, model_path in models_openvino.items():
    try:
        result = benchmark_openvino_inference(
            model_path=model_path,
            data_yaml=data_yaml,
            model_name=model_name,
            input_size=(640, 640),
            num_warmup=5,
            num_samples=100,
            device="CPU"
        )
        if result:
            results.append(result)
    except Exception as e:
        print(f"❌ Error benchmarking OpenVINO {model_name}: {e}")

# %%
# --- Benchmark NCNN ---
print("\n" + "="*60)
print("🔥 NCNN BENCHMARK")
print("="*60)

for model_name, model_dir in models_ncnn.items():
    try:
        param_path = os.path.join(model_dir, "model.ncnn.param")
        bin_path = os.path.join(model_dir, "model.ncnn.bin")
        
        if not (os.path.exists(param_path) and os.path.exists(bin_path)):
            print(f"⚠️  Missing NCNN model files for {model_name}, skipping.")
            continue
        
        result = benchmark_ncnn_inference(
            param_path=param_path,
            bin_path=bin_path,
            data_yaml=data_yaml,
            model_name=model_name,
            input_size=(640, 640),
            num_warmup=5,
            num_samples=100,
            use_gpu=False,
            num_threads=4
        )
        if result:
            results.append(result)
    except Exception as e:
        print(f"❌ Error benchmarking NCNN {model_name}: {e}")


print("\n" + "="*60)
print("✅ BENCHMARK SUMMARY")
print("="*60)

df = pd.DataFrame(results)

print(df.to_string(index=False))

# Save to CSV
save_path = "./Eval/FPS/detect_eval_results_fps_v5_8_11_custom_part2.csv"
df.to_csv(save_path, index=False)
print(f"\n💾 Saved results to: {save_path}")


🔥 OPENVINO BENCHMARK

🚀 Benchmarking OpenVINO: yolo5.xml
📐 Input size: (640, 640)
🖥️  Device: CPU
📊 Running 100 samples (after 5 warmup)

🔥 Warming up for 5 runs...

⏱️  Running timed inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 4.92 ms
   ⚙️  Inference : 64.72 ms
   📦 Postprocess: 0.46 ms
   ⏱️  Total: 70.11 ms → 14.26 FPS

🚀 Benchmarking OpenVINO: yolo8.xml
📐 Input size: (640, 640)
🖥️  Device: CPU
📊 Running 100 samples (after 5 warmup)

🔥 Warming up for 5 runs...

⏱️  Running timed inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 4.92 ms
   ⚙️  Inference : 67.03 ms
   📦 Postprocess: 0.47 ms
   ⏱️  Total: 72.42 ms → 13.81 FPS

🚀 Benchmarking OpenVINO: yolo11.xml
📐 Input size: (640, 640)
🖥️  Device: CPU
📊 Running 100 samples (after 5 warmup)

🔥 Warming up for 5 runs...

⏱️  Running timed inference on 100 test images...

📊 Average Timing (over 100 images):
   🧩 Preprocess: 4.88 ms
   ⚙️  Inference : 67.62 ms
